## Json기반으로 damage labeling.csv 생성

In [1]:
import os
import json
from glob import glob
from tqdm import tqdm
import pandas as pd

In [2]:
os.getcwd()

'C:\\Users\\SSAFY\\Desktop\\쏘카\\160. 차량파손 이미지 데이터\\01.데이터'

### 데이터 폴더의 최상단기준에서 실행


In [90]:
damage_images = glob('./2.Validation/2.라벨링데이터/damage/*.json')
damage_part_images = glob('./2.Validation/2.라벨링데이터/damage_part/*.json')

In [91]:
damage_df = pd.DataFrame(columns=("index", "Scratched", "Breakage", "Separated", "Crushed", "total_anns", "ran_var", "dataset"))

In [92]:
def cls_damage(data):
    file_name = data['images']['file_name']
    damage_dict = {"index": file_name, "Scratched" : 0, "Breakage" : 0, "Separated" : 0, "Crushed" : 0, "total_anns" : 0,"dataset" : ""}
    _sum = 0
    for i in range(len(data['annotations'])):
        damage_type = data['annotations'][i]['damage']
        damage_dict[damage_type] += 1
        _sum += 1
    damage_dict["total_anns"] = _sum
    return damage_dict

In [93]:
for i in tqdm(range(len(damage_images))):
    img = damage_images[i][2:]
    
    with open(f'./{img}', 'r') as f:
        data = json.load(f)
    damage_dict = cls_damage(data)
    
    if(1 <= i % 10 < 8):
        damage_dict['dataset'] = 'train'
    elif(i % 10 == 8 or i % 10 == 9):
        damage_dict['dataset'] = 'val'
    else:
        damage_dict['dataset'] = 'test'
    
    damage_df.loc[i] = damage_dict
damage_df.loc["총합", :] = damage_df.sum(axis = 0, numeric_only = True)

100%|███████████████████████████████████████████████████████████████████████████| 10404/10404 [00:20<00:00, 498.02it/s]


In [96]:
# dfframe셀 실수 -> 정수변경
damage_df = damage_df.astype({'Scratched' : 'int', 'Breakage' : 'int', 'Separated' : 'int', 'Crushed' : 'int'})

In [99]:
damage_df.head(10)

,index,Scratched,Breakage,Separated,Crushed,ran_var,dataset
0,0506233_sc-202337.jpg,0,1,1,0,NaN,test
1,0506234_sc-202337.jpg,0,0,3,0,NaN,train
2,0506235_sc-202337.jpg,1,0,2,0,NaN,train
3,0506236_sc-202337.jpg,1,0,1,0,NaN,train
4,0506237_sc-202337.jpg,2,0,1,0,NaN,train
5,0506238_as-3203576.jpg,2,0,0,0,NaN,train
6,0506239_as-3483186.jpg,0,0,1,0,NaN,train
7,0506240_as-7962812.jpg,1,1,1,1,NaN,train
8,0506241_sc-190131.jpg,2,0,1,0,NaN,val
9,0506245_sc-198049.jpg,1,0,0,1,NaN,val


In [100]:
damage_df.to_csv('./damage_labeling.csv', header=True, index=False)

### images
id  
width  
height  
filme_name

## annotations

id : 라벨링 식별자  
category_id : 사고아이디  
damage : 손상종류  
part : 차량부위  
level : 손상심각도(경미 손상, 소규모 손상, 대규모 손상, 교체수준)  
repair : 수리 방법

## categories 
id : 사고아이디  
supercategory_name : 차량크기